<a href="https://colab.research.google.com/github/wannasmile/colab_code_note/blob/main/BI0005.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from abc import ABCMeta
from abc import abstractmethod
from typing import Union

import numpy as np
import numpy.typing as npt
from scipy import stats
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.power import TTestIndPower


class BaseMetric:
    __metaclass__ = ABCMeta
    mde: float

    def __init__(self, mde: float, alternative: str):
        self.mde = mde
        self.alternative = alternative

    @property
    @abstractmethod
    def power_analysis_instance(self) -> Union[NormalIndPower, TTestIndPower]:
        raise NotImplementedError

    @property
    @abstractmethod
    def variance(self) -> float:
        raise NotImplementedError

    @staticmethod
    def check_positive(number: float, name: str) -> float:
        if number < 0:
            raise ValueError(f"Error: Please provide a positive number for {name}.")
        else:
            return number

    def generate_p_values(
        self, true_alt: npt.NDArray[np.bool_], sample_size: int, random_state: np.random.RandomState
    ) -> npt.NDArray[np.float_]:
        """
        This method simulates any registered metric's p-value. The output will
        later be applied to BH procedure

        Parameters:
            true_alt: A boolean array of shape (m hypotheses x replications).
            Each element represents whether the alternative hypothesis is true
            for an individual hypothesis sample_size: sample size used for simulations
            sample_size: an integer used to generate
            random_state: random state to generate fixed output for any given input


        Returns:
            p-value: A float array of shape (m hypotheses x replications) of
            simulated p-values
        """
        total_alt = true_alt.sum()
        total_null = true_alt.size - total_alt

        p_values = np.empty(true_alt.shape)
        p_values[true_alt] = self._generate_alt_p_values(total_alt, sample_size, random_state)
        p_values[~true_alt] = stats.uniform.rvs(0, 1, total_null, random_state=random_state)

        return p_values

    @abstractmethod
    def _generate_alt_p_values(
        self, size: int, sample_size: int, random_state: np.random.RandomState
    ) -> npt.NDArray[np.float_]:
        raise NotImplementedError


class BooleanMetric(BaseMetric):
    probability: float
    mde: float

    def __init__(
        self,
        probability: float,
        mde: float,
        alternative: str,
    ):
        super(BooleanMetric, self).__init__(mde, alternative)
        self.probability = self._check_probability(probability)

    @property
    def variance(self) -> float:
        return self.probability * (1 - self.probability)

    @property
    def power_analysis_instance(self) -> NormalIndPower:
        return NormalIndPower()

    @staticmethod
    def _check_probability(probability: float) -> float:
        if 0 <= probability <= 1:
            return probability
        else:
            raise ValueError("Error: Please provide a float between 0 and 1 for probability.")

    def _generate_alt_p_values(
        self, size: int, sample_size: int, random_state: np.random.RandomState
    ) -> npt.NDArray[np.float_]:
        effect_size = self.mde / np.sqrt(2 * self.variance / sample_size)
        z_alt = stats.norm.rvs(loc=effect_size, size=size, random_state=random_state)
        p_values: npt.NDArray[np.float_] = stats.norm.sf(np.abs(z_alt))
        if self.alternative == "two-sided":
            p_values *= 2
        return p_values


class NumericMetric(BaseMetric):
    mde: float

    def __init__(
        self,
        variance: float,
        mde: float,
        alternative: str,
    ):
        super(NumericMetric, self).__init__(mde, alternative)
        self._variance = self.check_positive(variance, "variance")

    @property
    def variance(self) -> float:
        return self._variance

    @property
    def power_analysis_instance(self) -> TTestIndPower:
        return TTestIndPower()

    def _generate_alt_p_values(
        self, size: int, sample_size: int, random_state: np.random.RandomState
    ) -> npt.NDArray[np.float_]:
        nc = np.sqrt(sample_size / 2 / self.variance) * self.mde
        t_alt = stats.nct.rvs(nc=nc, df=2 * (sample_size - 1), size=size, random_state=random_state)
        p_values: npt.NDArray[np.float_] = stats.t.sf(np.abs(t_alt), 2 * (sample_size - 1))
        # Todo: use accurate p-value calculation due to nct's asymmetric distribution
        if self.alternative == "two-sided":
            p_values *= 2
        return p_values


class RatioMetric(BaseMetric):
    numerator_mean: float
    numerator_variance: float
    denominator_mean: float
    denominator_variance: float
    covariance: float

    def __init__(
        self,
        numerator_mean: float,
        numerator_variance: float,
        denominator_mean: float,
        denominator_variance: float,
        covariance: float,
        mde: float,
        alternative: str,
    ):
        super(RatioMetric, self).__init__(mde, alternative)
        # TODO: add check for Cauchy-Schwarz inequality
        self.numerator_mean = numerator_mean
        self.numerator_variance = self.check_positive(numerator_variance, "numerator variance")
        self.denominator_mean = denominator_mean
        self.denominator_variance = self.check_positive(denominator_variance, "denominator variance")
        self.covariance = covariance

    @property
    def variance(self) -> float:
        variance = (
            self.numerator_variance / self.denominator_mean**2
            + self.denominator_variance * self.numerator_mean**2 / self.denominator_mean**4
            - 2 * self.covariance * self.numerator_mean / self.denominator_mean**3
        )

        return variance

    @property
    def power_analysis_instance(self) -> NormalIndPower:
        return NormalIndPower()

    def _generate_alt_p_values(
        self, size: int, sample_size: int, random_state: np.random.RandomState
    ) -> npt.NDArray[np.float_]:
        effect_size = self.mde / np.sqrt(2 * self.variance / sample_size)
        z_alt = stats.norm.rvs(loc=effect_size, size=size, random_state=random_state)
        p_values: npt.NDArray[np.float_] = stats.norm.sf(np.abs(z_alt))
        if self.alternative == "two-sided":
            p_values *= 2
        return p_values

In [2]:
from typing import List

import numpy as np
import numpy.typing as npt
from statsmodels.stats.multitest import multipletests

#from sample_size.metrics import BaseMetric

DEFAULT_REPLICATION: int = 400
DEFAULT_EPSILON: float = 0.01
DEFAULT_MAX_RECURSION: int = 20


class MultipleTestingMixin:
    """
    This class calculates sample size required under the case of multiple testing

    Attributes:
    metrics: a list of BaseMetric registered by users
    variants: number of variants, including control
    alpha: statistical significance
    power: average power, calculated as #correct rejections/#true alternative hypotheses

    """

    metrics: List[BaseMetric]
    alpha: float
    power: float
    variants: int

    def get_multiple_sample_size(
        self,
        lower: float,
        upper: float,
        random_state: np.random.RandomState,
        depth: int = 0,
        replication: int = DEFAULT_REPLICATION,
        epsilon: float = DEFAULT_EPSILON,
        max_recursion_depth: int = DEFAULT_MAX_RECURSION,
    ) -> int:
        """
        This method finds minimum required sample size per cohort that generates
        average power higher than required

        Attributes:
            lower: lower bound of sample size search
            upper: upper bound of sample size search
            depth: number of recursions
            replication: number of Monte Carlo simulations to calculate empirical power
            epsilon: absolute difference between our estimate for power and desired power
                needed before we will return
            max_recursion_depth: how many recursive calls can be made before the
                search is abandoned

        Returns
            minimum required sample size per cohort
        """

        if depth > max_recursion_depth:
            raise RecursionError(f"Couldn't find a sample size that satisfies the power you requested: {self.power}")

        candidate = int(np.sqrt(lower * upper))
        expected_power = self._expected_average_power(candidate, random_state, replication)
        if np.isclose(self.power, expected_power, atol=epsilon):
            return candidate
        elif lower == upper:
            raise RecursionError(f"Couldn't find a sample size that satisfies the power you requested: {self.power}")

        if expected_power > self.power:
            return self.get_multiple_sample_size(lower, candidate, random_state, depth + 1)
        else:
            return self.get_multiple_sample_size(candidate, upper, random_state, depth + 1)

    def _expected_average_power(
        self, sample_size: int, random_state: np.random.RandomState, replication: int = DEFAULT_REPLICATION
    ) -> float:
        """
        This method calculates expected average power of multiple testings. For each possible number of true null
        hypothesis, we simulate each metric/treatment variant's test statistics and calculate their p-values,
        then calculate expected average power = number of True rejection/ true alternative hypotheses

        Attributes:
        sample size: determines the variance/ degrees of freedom of the distribution we sample test statistics from
        replication: number of times we repeat the simulation process

        Returns value expected average power
        """
        true_alt_count = 0.0
        true_discovery_count = 0.0

        # a metric for each test we would conduct
        metrics = self.metrics * (self.variants - 1)

        def fdr_bh(a: npt.NDArray[np.float_]) -> npt.NDArray[np.bool_]:
            rejected: npt.NDArray[np.bool_] = multipletests(a, alpha=self.alpha, method="fdr_bh")[0]
            return rejected

        for num_true_alt in range(1, len(metrics) + 1):
            true_alt = np.array([random_state.permutation(len(metrics)) < num_true_alt for _ in range(replication)]).T
            p_values = []
            for i, m in enumerate(metrics):
                p_values.append(m.generate_p_values(true_alt[i], sample_size, random_state))

            rejected = np.apply_along_axis(fdr_bh, 0, np.array(p_values))  # type: ignore[no-untyped-call]

            true_discoveries = rejected & true_alt

            true_discovery_count += true_discoveries.sum()
            true_alt_count += true_alt.sum()

        avg_power = true_discovery_count / true_alt_count

        return avg_power

In [3]:
metrics_schema_json_file = open('metrics_schema.json', 'w')

metrics_schema_json = '''
{
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "metric_type": {
                "type": "string",
                "enum": ["boolean", "numeric", "ratio"]
            }
        },
        "allOf": [
            {
                "if": {
                    "properties": {
                        "metric_type": {"const": "boolean"}
                    }
                },
                "then": {
                    "properties": {
                        "metric_metadata": {
                            "type": "object",
                            "properties": {
                                "alternative": {"type": "string"},
                                "mde": {"type": "number"},
                                "probability": {"type": "number"}
                            },
                            "required": ["mde", "probability"]
                        }
                    }
                }
            },
            {
                "if": {
                    "properties": {
                        "metric_type": {"const": "numeric"}
                    }
                },
                "then": {
                    "properties": {
                        "metric_metadata": {
                            "type": "object",
                            "properties": {
                                "alternative": {"type": "string"},
                                "mde": {"type": "number"},
                                "variance": {"type": "number"}
                            },
                            "required": ["mde", "variance"]
                        }
                    }
                }
            },
            {
                "if": {
                    "properties": {
                        "metric_type": {"const": "ratio"}
                    }
                },
                "then": {
                    "properties": {
                        "metric_metadata": {
                            "type": "object",
                            "properties": {
                                "alternative": {"type": "string"},
                                "mde": {"type": "number"},
                                "numerator_mean": {"type": "number"},
                                "numerator_variance": {"type": "number"},
                                "denominator_mean": {"type": "number"},
                                "denominator_variance": {"type": "number"},
                                "covariance": {"type": "number"}
                            },
                            "required": [
                                "mde",
                                "numerator_mean",
                                "numerator_variance",
                                "denominator_mean",
                                "denominator_variance",
                                "covariance"
                            ]
                        }
                    }
                }
            }
        ],
        "required": ["metric_type", "metric_metadata"]
    },
    "minItems": 1
}
'''

metrics_schema_json_file.write(metrics_schema_json)

metrics_schema_json_file.close()

In [4]:
import json
from pathlib import Path
from typing import Any
from typing import Dict
from typing import List

import numpy as np
from jsonschema import validate

#from sample_size.metrics import BaseMetric
#from sample_size.metrics import BooleanMetric
#from sample_size.metrics import NumericMetric
#from sample_size.metrics import RatioMetric
#from sample_size.multiple_testing import MultipleTestingMixin

DEFAULT_ALPHA = 0.05
DEFAULT_POWER = 0.8
DEFAULT_VARIANTS = 2
RANDOM_STATE = np.random.RandomState(1)
STATE = RANDOM_STATE.get_state()

schema_file_path = Path("metrics_schema.json")
#schema_file_path = Path(Path(__file__).parent, "metrics_schema.json")
with open(str(schema_file_path), "r") as schema_file:
    METRICS_SCHEMA = json.load(schema_file)


class SampleSizeCalculator(MultipleTestingMixin):
    """
    This class is to calculate sample size based on metric type

    Attributes:
    alpha: statistical significance
    power: statistical power

    """

    def __init__(self, alpha: float = DEFAULT_ALPHA, variants: int = DEFAULT_VARIANTS, power: float = DEFAULT_POWER):
        self.alpha = alpha
        self.power = power
        self.metrics: List[BaseMetric] = []
        self.variants: int = variants

    def _get_single_sample_size(self, metric: BaseMetric, alpha: float) -> float:
        effect_size = metric.mde / float(np.sqrt(metric.variance))
        power_analysis = metric.power_analysis_instance
        sample_size = int(
            power_analysis.solve_power(
                effect_size=effect_size,
                alpha=alpha,
                power=self.power,
                ratio=1,
                alternative=metric.alternative,
            )
        )
        return sample_size

    def get_sample_size(self) -> float:
        if len(self.metrics) * (self.variants - 1) < 2:
            return self._get_single_sample_size(self.metrics[0], self.alpha)

        num_tests = len(self.metrics) * (self.variants - 1)
        lower = min([self._get_single_sample_size(metric, self.alpha) for metric in self.metrics])
        upper = max([self._get_single_sample_size(metric, self.alpha / num_tests) for metric in self.metrics])

        RANDOM_STATE.set_state(STATE)
        return self.get_multiple_sample_size(lower, upper, RANDOM_STATE)

    def register_metrics(self, metrics: List[Dict[str, Any]]) -> None:
        METRIC_REGISTER_MAP = {
            "boolean": BooleanMetric,
            "numeric": NumericMetric,
            "ratio": RatioMetric,
        }

        validate(instance=metrics, schema=METRICS_SCHEMA)

        for metric in metrics:
            metric_class = METRIC_REGISTER_MAP[metric["metric_type"]]
            registered_metric = metric_class(**metric["metric_metadata"])
            self.metrics.append(registered_metric)

In [5]:
!pip install parameterized

In [6]:
import unittest
from itertools import combinations_with_replacement as combos
from itertools import product
from unittest.mock import MagicMock
from unittest.mock import patch

import numpy as np
from numpy.testing import assert_array_equal
from parameterized import parameterized
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.power import TTestIndPower

#from sample_size.metrics import BaseMetric
#from sample_size.metrics import BooleanMetric
#from sample_size.metrics import NumericMetric
#from sample_size.metrics import RatioMetric
#from sample_size.sample_size_calculator import RANDOM_STATE

ALTERNATIVE = "two-sided"
TEST_ALTERNATIVES = ("two-sided", "smaller", "larger")


class DummyMetric(BaseMetric):
    def power_analysis_instance(self):
        return MagicMock()

    def variance(self):
        return MagicMock()

    def _generate_alt_p_values(self, size, sample_size, RANDOM_STATE):
        return MagicMock()


class BaseMetricTestCase(unittest.TestCase):
    def test_check_positive(self):
        test_negative_number = -10
        test_name = "test"

        with self.assertRaises(Exception) as context:
            BaseMetric.check_positive(test_negative_number, test_name)

        self.assertEqual(
            str(context.exception),
            f"Error: Please provide a positive number for {test_name}.",
        )

    @parameterized.expand([(np.array(c),) for r in range(2, 5) for c in combos([True, False], r)])
    @patch("sample_size.metrics.stats")
    @patch("tests.sample_size.test_metrics.DummyMetric._generate_alt_p_values")
    def test_generate_p_values(self, true_alt, mock_alt_p_values, mock_stats):
        mde = 0.5
        sample_size = 10

        null_p_value = 1
        alt_p_value = 0

        mock_alt_p_values.side_effect = lambda size, __, random_state: np.array([alt_p_value] * size)
        mock_stats.uniform.rvs.side_effect = lambda _, __, size, random_state: np.array([null_p_value] * size)

        metric = DummyMetric(mde, ALTERNATIVE)

        p_values = metric.generate_p_values(true_alt, sample_size, RANDOM_STATE)

        mock_alt_p_values.assert_called_once()
        mock_stats.uniform.rvs.assert_called_once()

        assert_array_equal(p_values, np.where(true_alt, alt_p_value, null_p_value))


class BooleanMetricTestCase(unittest.TestCase):
    def setUp(self):
        self.DEFAULT_ALTERNATIVE = ALTERNATIVE
        self.DEFAULT_MDE = 0.01
        self.DEFAULT_PROBABILITY = 0.05
        self.DEFAULT_MOCK_VARIANCE = 99

    @patch("sample_size.metrics.BooleanMetric._check_probability")
    @patch("sample_size.metrics.BooleanMetric.variance")
    def test_boolean_metric_constructor_sets_params(self, mock_variance, mock_check_probability):
        mock_variance.__get__ = MagicMock(return_value=self.DEFAULT_MOCK_VARIANCE)
        mock_check_probability.return_value = self.DEFAULT_PROBABILITY
        boolean = BooleanMetric(self.DEFAULT_PROBABILITY, self.DEFAULT_MDE, self.DEFAULT_ALTERNATIVE)

        mock_check_probability.assert_called_once_with(self.DEFAULT_PROBABILITY)
        self.assertEqual(boolean.probability, self.DEFAULT_PROBABILITY)
        self.assertEqual(boolean.variance, self.DEFAULT_MOCK_VARIANCE)
        self.assertEqual(boolean.mde, self.DEFAULT_MDE)
        self.assertIsInstance(boolean.power_analysis_instance, NormalIndPower)

    def test_boolean_metric_variance(self):
        boolean = BooleanMetric(self.DEFAULT_PROBABILITY, self.DEFAULT_MDE, self.DEFAULT_ALTERNATIVE)

        self.assertEqual(boolean.variance, 0.0475)

    def test_boolean_metric_get_probability(self):
        probability = BooleanMetric._check_probability(self.DEFAULT_PROBABILITY)

        self.assertEqual(probability, self.DEFAULT_PROBABILITY)

    def test_boolean_metric_get_probability_too_large(self):
        test_probability = 5

        with self.assertRaises(Exception) as context:
            BooleanMetric._check_probability(test_probability)

        self.assertEqual(
            str(context.exception),
            "Error: Please provide a float between 0 and 1 for probability.",
        )

    def test_boolean_metric_get_probability_too_small(self):
        test_probability = -0.1

        with self.assertRaises(Exception) as context:
            BooleanMetric._check_probability(test_probability)

        self.assertEqual(
            str(context.exception),
            "Error: Please provide a float between 0 and 1 for probability.",
        )

    @parameterized.expand(product((1, 2, 10), (2, 10), TEST_ALTERNATIVES))
    @patch("sample_size.metrics.BooleanMetric.variance")
    @patch("scipy.stats.norm")
    def test_boolean__generate_alt_p_values(self, size, sample_size, alternative, mock_norm, mock_variance):
        p_value_generator = mock_norm.sf
        p_values = MagicMock()
        mock_norm.rvs.return_value = -ord("🌮")
        p_value_generator.return_value = p_values
        mock_variance.__get__ = MagicMock(return_value=self.DEFAULT_MOCK_VARIANCE)

        metric = BooleanMetric(self.DEFAULT_PROBABILITY, self.DEFAULT_MDE, alternative)
        p = metric._generate_alt_p_values(size, sample_size, RANDOM_STATE)

        effect_sample_size = self.DEFAULT_MDE / np.sqrt(2 * self.DEFAULT_MOCK_VARIANCE / sample_size)
        mock_norm.rvs.assert_called_once_with(loc=effect_sample_size, size=size, random_state=RANDOM_STATE)
        mock_norm.sf.assert_called_once_with(np.abs(mock_norm.rvs.return_value))
        expected_p_values = p_values if alternative != "two-sided" else 2 * p_values
        assert_array_equal(p, expected_p_values)


class NumericMetricTestCase(unittest.TestCase):
    def setUp(self):
        self.DEFAULT_MDE = 5
        self.DEFAULT_VARIANCE = 5000
        self.DEFAULT_ALTERNATIVE = ALTERNATIVE

    def test_numeric_metric_constructor_sets_params(self):
        numeric = NumericMetric(self.DEFAULT_VARIANCE, self.DEFAULT_MDE, self.DEFAULT_ALTERNATIVE)

        self.assertEqual(numeric.variance, self.DEFAULT_VARIANCE)
        self.assertEqual(numeric.mde, self.DEFAULT_MDE)
        self.assertEqual(numeric.alternative, self.DEFAULT_ALTERNATIVE)
        self.assertIsInstance(numeric.power_analysis_instance, TTestIndPower)

    @parameterized.expand(product((1, 2, 10), (2, 10), TEST_ALTERNATIVES))
    @patch("sample_size.metrics.NumericMetric.variance")
    @patch("scipy.stats.nct")
    @patch("scipy.stats.t")
    def test_numeric__generate_alt_p_values(self, size, sample_size, alternative, mock_t, mock_nct, mock_variance):
        p_value_generator = mock_t.sf
        p_values = MagicMock()
        mock_nct.rvs.return_value = -ord("🌮")
        p_value_generator.return_value = p_values
        mock_variance.__get__ = MagicMock(return_value=self.DEFAULT_VARIANCE)

        metric = NumericMetric(self.DEFAULT_VARIANCE, self.DEFAULT_MDE, alternative)
        p = metric._generate_alt_p_values(size, sample_size, RANDOM_STATE)

        effect_sample_size = np.sqrt(sample_size / 2 / self.DEFAULT_VARIANCE) * self.DEFAULT_MDE
        df = 2 * (sample_size - 1)
        mock_nct.rvs.assert_called_once_with(nc=effect_sample_size, df=df, size=size, random_state=RANDOM_STATE)
        mock_t.sf.assert_called_once_with(np.abs(mock_nct.rvs.return_value), df)
        expected_p_values = p_values if alternative != "two-sided" else 2 * p_values
        assert_array_equal(p, expected_p_values)


class RatioMetricTestCase(unittest.TestCase):
    def setUp(self):
        self.DEFAULT_MDE = 5
        self.DEFAULT_NUMERATOR_MEAN = 2000
        self.DEFAULT_NUMERATOR_VARIANCE = 100000
        self.DEFAULT_DENOMINATOR_MEAN = 200
        self.DEFAULT_DENOMINATOR_VARIANCE = 2000
        self.DEFAULT_COVARIANCE = 5000
        self.DEFAULT_VARIANCE = 99
        self.DEFAULT_ALTERNATIVE = ALTERNATIVE

    @patch("sample_size.metrics.RatioMetric.variance")
    def test_ratio_metric_constructor_sets_params(self, mock_variance):
        mock_variance.__get__ = MagicMock(return_value=self.DEFAULT_VARIANCE)
        ratio = RatioMetric(
            self.DEFAULT_NUMERATOR_MEAN,
            self.DEFAULT_NUMERATOR_VARIANCE,
            self.DEFAULT_DENOMINATOR_MEAN,
            self.DEFAULT_DENOMINATOR_VARIANCE,
            self.DEFAULT_COVARIANCE,
            self.DEFAULT_MDE,
            self.DEFAULT_ALTERNATIVE,
        )

        self.assertEqual(ratio.numerator_mean, self.DEFAULT_NUMERATOR_MEAN)
        self.assertEqual(ratio.numerator_variance, self.DEFAULT_NUMERATOR_VARIANCE)
        self.assertEqual(ratio.denominator_mean, self.DEFAULT_DENOMINATOR_MEAN)
        self.assertEqual(ratio.denominator_variance, self.DEFAULT_DENOMINATOR_VARIANCE)
        self.assertEqual(ratio.covariance, self.DEFAULT_COVARIANCE)
        self.assertEqual(ratio.variance, self.DEFAULT_VARIANCE)
        self.assertEqual(ratio.mde, self.DEFAULT_MDE)
        self.assertIsInstance(ratio.power_analysis_instance, NormalIndPower)

    def test_ratio_metric_variance(self):
        ratio = RatioMetric(
            self.DEFAULT_NUMERATOR_MEAN,
            self.DEFAULT_NUMERATOR_VARIANCE,
            self.DEFAULT_DENOMINATOR_MEAN,
            self.DEFAULT_DENOMINATOR_VARIANCE,
            self.DEFAULT_COVARIANCE,
            self.DEFAULT_MDE,
            self.DEFAULT_ALTERNATIVE,
        )

        self.assertEqual(ratio.variance, 5.0)

    @parameterized.expand(product((1, 2, 10), (2, 10), TEST_ALTERNATIVES))
    @patch("sample_size.metrics.RatioMetric.variance")
    @patch("scipy.stats.norm")
    def test_ratio__generate_alt_p_values(self, size, sample_size, alternative, mock_norm, mock_variance):
        p_value_generator = mock_norm.sf
        p_values = MagicMock()
        mock_norm.rvs.return_value = -ord("🌮")
        p_value_generator.return_value = p_values
        mock_variance.__get__ = MagicMock(return_value=self.DEFAULT_VARIANCE)

        metric = RatioMetric(
            self.DEFAULT_NUMERATOR_MEAN,
            self.DEFAULT_NUMERATOR_VARIANCE,
            self.DEFAULT_DENOMINATOR_MEAN,
            self.DEFAULT_DENOMINATOR_VARIANCE,
            self.DEFAULT_COVARIANCE,
            self.DEFAULT_MDE,
            self.DEFAULT_ALTERNATIVE,
        )

        p = metric._generate_alt_p_values(size, sample_size, RANDOM_STATE)

        effect_sample_size = self.DEFAULT_MDE / np.sqrt(2 * self.DEFAULT_VARIANCE / sample_size)
        mock_norm.rvs.assert_called_once_with(loc=effect_sample_size, size=size, random_state=RANDOM_STATE)
        mock_norm.sf.assert_called_once_with(np.abs(mock_norm.rvs.return_value))
        expected_p_values = p_values if alternative != "two-sided" else 2 * p_values
        assert_array_equal(p, expected_p_values)

In [7]:
import unittest
from unittest.mock import call
from unittest.mock import patch

from parameterized import parameterized

#from sample_size.metrics import BooleanMetric
#from sample_size.metrics import NumericMetric
#from sample_size.metrics import RatioMetric
#from sample_size.sample_size_calculator import DEFAULT_ALPHA
#from sample_size.sample_size_calculator import DEFAULT_POWER
#from sample_size.sample_size_calculator import DEFAULT_VARIANTS
#from sample_size.sample_size_calculator import RANDOM_STATE
#from sample_size.sample_size_calculator import SampleSizeCalculator
#from tests.sample_size.test_metrics import ALTERNATIVE


class SampleSizeCalculatorTestCase(unittest.TestCase):
    def test_sample_size_calculator_constructor_sets_params(self):
        test_alpha = 0.1
        test_variants = 2
        test_power = 0.9
        calculator = SampleSizeCalculator(
            test_alpha,
            test_variants,
            test_power,
        )

        self.assertEqual(calculator.alpha, test_alpha)
        self.assertEqual(calculator.power, test_power)
        self.assertEqual(calculator.metrics, [])

    def test_sample_size_calculator_constructor_sets_params_with_default_params(self):
        calculator = SampleSizeCalculator()

        self.assertEqual(calculator.alpha, DEFAULT_ALPHA)
        self.assertEqual(calculator.variants, DEFAULT_VARIANTS)
        self.assertEqual(calculator.power, DEFAULT_POWER)
        self.assertEqual(calculator.metrics, [])

    @patch("statsmodels.stats.power.NormalIndPower.solve_power")
    def test_get_single_sample_size_normal(self, mock_solve_power):
        test_probability = 0.05
        test_mde = 0.02
        test_sample_size = 2000
        test_metric = BooleanMetric(
            test_probability,
            test_mde,
            ALTERNATIVE,
        )
        mock_solve_power.return_value = test_sample_size

        calculator = SampleSizeCalculator()
        sample_size = calculator._get_single_sample_size(test_metric, calculator.alpha)

        self.assertEqual(sample_size, test_sample_size)
        mock_solve_power.assert_called_once_with(
            effect_size=0.09176629354822471,
            alpha=DEFAULT_ALPHA,
            power=DEFAULT_POWER,
            ratio=1,
            alternative="two-sided",
        )

    @patch("statsmodels.stats.power.TTestIndPower.solve_power")
    def test_get_single_sample_size_ttest(self, mock_solve_power):
        test_variance = 1000
        test_mde = 5
        test_sample_size = 2000
        test_metric = NumericMetric(
            test_variance,
            test_mde,
            ALTERNATIVE,
        )
        mock_solve_power.return_value = test_sample_size
        calculator = SampleSizeCalculator()

        sample_size = calculator._get_single_sample_size(test_metric, calculator.alpha)

        self.assertEqual(sample_size, test_sample_size)
        mock_solve_power.assert_called_once_with(
            effect_size=0.15811388300841897,
            alpha=DEFAULT_ALPHA,
            power=DEFAULT_POWER,
            ratio=1,
            alternative="two-sided",
        )

    @parameterized.expand(
        [
            ("boolean", {"probability": 0.05, "mde": 0.02, "alternative": "two-sided"}),
            ("numeric", {"variance": 500, "mde": 5, "alternative": "smaller"}),
            (
                "ratio",
                {
                    "numerator_mean": 2000,
                    "numerator_variance": 100000,
                    "denominator_mean": 200,
                    "denominator_variance": 2000,
                    "covariance": 5000,
                    "mde": 10,
                    "alternative": "larger",
                },
            ),
        ]
    )
    @patch("sample_size.sample_size_calculator.SampleSizeCalculator.get_multiple_sample_size")
    @patch("sample_size.sample_size_calculator.SampleSizeCalculator._get_single_sample_size")
    def test_get_sample_size_single(
        self, metric_type, metadata, mock_get_single_sample_size, mock_get_multiple_sample_size
    ):
        test_metric_type = metric_type
        test_sample_size = 2000
        mock_get_single_sample_size.return_value = test_sample_size

        test_mde = metadata["mde"]
        test_metric_metadata = metadata
        calculator = SampleSizeCalculator()
        calculator.register_metrics([{"metric_type": test_metric_type, "metric_metadata": test_metric_metadata}])

        sample_size = calculator.get_sample_size()

        self.assertEqual(sample_size, test_sample_size)
        mock_get_multiple_sample_size.assert_not_called()
        mock_get_single_sample_size.assert_called_once()
        self.assertEqual(mock_get_single_sample_size.call_args[0][0], calculator.metrics[0])
        self.assertEqual(mock_get_single_sample_size.call_args[0][0].mde, test_mde)

    @patch("sample_size.sample_size_calculator.SampleSizeCalculator.get_multiple_sample_size")
    @patch("sample_size.sample_size_calculator.SampleSizeCalculator._get_single_sample_size")
    def test_get_sample_size_multiple(self, mock_get_single_sample_size, mock_get_multiple_sample_size):
        test_metric_type = "boolean"
        test_sample_size = 2000
        mock_get_multiple_sample_size.return_value = test_sample_size
        mock_get_single_sample_size.return_value = test_sample_size
        test_metric_metadata = {"probability": 0.05, "mde": 0.02, "alternative": ALTERNATIVE}
        calculator = SampleSizeCalculator()
        calculator.register_metrics(
            [
                {"metric_type": test_metric_type, "metric_metadata": test_metric_metadata},
                {"metric_type": test_metric_type, "metric_metadata": test_metric_metadata},
            ]
        )

        sample_size = calculator.get_sample_size()

        self.assertEqual(sample_size, test_sample_size)
        self.assertEqual(mock_get_single_sample_size.call_count, 4)
        mock_get_single_sample_size.assert_has_calls(
            [
                call(calculator.metrics[0], calculator.alpha),
                call(calculator.metrics[1], calculator.alpha),
                call(calculator.metrics[0], calculator.alpha / 2),
                call(calculator.metrics[1], calculator.alpha / 2),
            ]
        )
        mock_get_multiple_sample_size.assert_called_once_with(test_sample_size, test_sample_size, RANDOM_STATE)

    # TODO: parameterize register metric functions
    def test_register_metric_boolean(self):
        test_metric_type = "boolean"
        test_probability = 0.05
        test_mde = 0.02
        test_metric_metadata = {"probability": test_probability, "mde": test_mde, "alternative": "larger"}

        calculator = SampleSizeCalculator()
        calculator.register_metrics([{"metric_type": test_metric_type, "metric_metadata": test_metric_metadata}])
        self.assertIsInstance(calculator.metrics[0], BooleanMetric)
        self.assertEqual(len(calculator.metrics), 1)
        self.assertEqual(calculator.metrics[0].variance, 0.0475)
        self.assertEqual(calculator.metrics[0].mde, test_mde)

        calculator.register_metrics([{"metric_type": test_metric_type, "metric_metadata": test_metric_metadata}])
        self.assertEqual(len(calculator.metrics), 2)

    def test_register_metric_numeric(self):
        test_metric_type = "numeric"
        test_variance = 5000.0
        test_mde = 5.0
        test_metric_metadata = {"variance": test_variance, "mde": test_mde, "alternative": "two-sided"}

        calculator = SampleSizeCalculator()
        calculator.register_metrics([{"metric_type": test_metric_type, "metric_metadata": test_metric_metadata}])
        self.assertIsInstance(calculator.metrics[0], NumericMetric)
        self.assertEqual(len(calculator.metrics), 1)
        self.assertEqual(calculator.metrics[0].variance, test_variance)
        self.assertEqual(calculator.metrics[0].mde, test_mde)

        calculator.register_metrics([{"metric_type": test_metric_type, "metric_metadata": test_metric_metadata}])
        self.assertEqual(len(calculator.metrics), 2)

    def test_register_metric_ratio(self):
        test_metric_type = "ratio"
        test_numerator_mean = 2000.0
        test_numerator_variance = 100000.0
        test_denominator_mean = 200.0
        test_denominator_variance = 2000.0
        test_covariance = 5000.0
        test_mde = 5.0
        test_variance = 5
        test_metric_metadata = {
            "numerator_mean": test_numerator_mean,
            "numerator_variance": test_numerator_variance,
            "denominator_mean": test_denominator_mean,
            "denominator_variance": test_denominator_variance,
            "covariance": test_covariance,
            "mde": test_mde,
            "alternative": "smaller",
        }

        calculator = SampleSizeCalculator()
        calculator.register_metrics([{"metric_type": test_metric_type, "metric_metadata": test_metric_metadata}])
        self.assertIsInstance(calculator.metrics[0], RatioMetric)
        self.assertEqual(len(calculator.metrics), 1)
        self.assertEqual(calculator.metrics[0].variance, test_variance)
        self.assertEqual(calculator.metrics[0].mde, test_mde)

        calculator.register_metrics([{"metric_type": test_metric_type, "metric_metadata": test_metric_metadata}])
        self.assertEqual(len(calculator.metrics), 2)

    def test_register_metric_invalid_metadata(self):
        test_metric_type = "numeric"

        calculator = SampleSizeCalculator()
        with self.assertRaises(Exception):
            calculator.register_metrics([{"metric_type": test_metric_type}])

In [8]:
tester = SampleSizeCalculatorTestCase()
tester.test_sample_size_calculator_constructor_sets_params()

> Baseline Conversion Rate

Your control group's expected conversion rate.



> Minimum Detectable Effect

The minimum relative change in conversion rate you would like to be able to detect.


> Statistical Significance

95% is an accepted standard for statistical significance, although Optimizely allows you to set your own threshold for significance based on your risk tolerance.



第一类错误：原假设为真的时候，拒绝原假设的概率。

α：假设AB没有差异时，观察到有差异的概率

> Significance level α

Percent of the time a difference will be detected, assuming one does NOT exist


第二类错误：原假设为假的时候，接受原假设的概率。

β：假设AB有差异时，观察到没有差异的概率

> Statistical power 1−β

Percent of the time the minimum effect size will be detected, assuming it exists



# Evan's Awesome A/B Tools:

## Sample Size Calculator

### Question: How many subjects are needed for an A/B test?

Baseline conversion rate


Minimum Detectable Effect:
The Minimum Detectable Effect is the smallest effect that will be detected (1-β)% of the time.


Statistical power 1−β:
Percent of the time the minimum effect size will be detected, assuming it exists.

Significance level α:
Percent of the time a difference will be detected, assuming one does NOT exist.


In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from scipy.stats import norm


def compute_sample_size(p0, mde, alpha=0.05, beta=0.2, tails="Two"):
    """
    Returns the sample size for a two-tailed AB test comparing conversion
    rates.

    The sample size equation is for binomial distributions only.

    Parameters
    ----------
    p0 : float
        Baseline conversion rate

    mde : float or int
        Minimum detectable effect. This is the 'sensitivity' of the test or
        the relative difference in conversion rates that you want to be able
        to detect.

    alpha : float
        The chances of a Type I error. Tests are normally run to a 95%
        significance meaning an alpha of 1 - 0.95 = 0.05. Default = 0.05.

    beta : float
        The chances of a Type II error. For sample sizing, a beta of 0.2 is
        acceptable and provides the test with 80% statistical power as is
        standard.

    tails : str
        One or two tails to specify what type of hypothesis test this is.

    Returns
    -------
    Minimum number of observations required per variant.
    """

    # Conditional alpha value based on whether one or two tail test
    if tails == "Two":
        computed_alpha = alpha / 2
    else:
        computed_alpha = alpha

    p1 = p0 * (1 + mde)
    N = (
        (norm.ppf(1 - computed_alpha) + norm.ppf(1 - beta)) ** 2
        * (p0 * (1 - p0) + p1 * (1 - p1))
        / ((p0 - p1) ** 2)
    )
    return int(N)

In [10]:
print(compute_sample_size(0.89, 0.0011, 0.05, 0.2, "Two"))

1597187


In [11]:
print(compute_sample_size(0.89, 0.0011, 0.05, 0.2, "One"))

1258103


## Chi-Squared Test

### Question: Does the rate of success differ across two groups?

If the experiment is repeated many times, the confidence level is the percent of the time each sample's success rate will fall within the reported confidence interval.

It is also the percent of the time no difference will be detected between the two groups, assuming no difference exists.

In [12]:
from statsmodels.stats.proportion import proportions_ztest as ztest
import numpy as np

ztest(count=np.array([3000,3200]), nobs=np.array([10000,10000]))

(-3.057803726183795, 0.0022296556161908814)

对照组：参与人数 10000，转化人数 3000

实验组：参与人数 10000，转化人数 3200

显著性水平：0.05


第1个值为z分数，第2个值为p值。
p值<0.05，从而可知两组存在显著差异，实验结果提升是显著的。


In [13]:
ztest(count=np.array([3200,3000]), nobs=np.array([10000,10000]))

(3.057803726183795, 0.0022296556161908814)

In [14]:
ztest(count=np.array([320,3000]), nobs=np.array([1000,10000]))

(1.3136409747118007, 0.18896705292826543)

In [15]:
ztest(count=np.array([125810*(0.89-0.0011),1258103*0.89]), nobs=np.array([125810,1258103]))

(-1.1884740633049435, 0.23464669236156277)

In [16]:
ztest(count=np.array([1258103*(0.89-0.0011),1258103*0.89]), nobs=np.array([1258103,1258103]))

(-2.782246745764699, 0.005398397957847781)

In [17]:
ztest(count=np.array([2000000*(0.89-0.0011),2000000*0.89]), nobs=np.array([2000000,2000000]))

(-3.5079431131165877, 0.0004515856033560191)

In [18]:
r=1
z,p = ztest(count=np.array([1258103*(0.89-0.0011),1258103*0.89]), nobs=np.array([1258103,1258103]))
while p<=0.05:
  r=r-0.1
  z,p = ztest(count=np.array([1258103*r*(0.89-0.0011),1258103*0.89]), nobs=np.array([1258103*r,1258103]))

print(r)

0.30000000000000016


In [19]:
ztest(count=np.array([1258103*0.4*(0.89-0.0011),1258103*0.89]), nobs=np.array([1258103*0.4,1258103]))

(-2.105147713673478, 0.035278451987355554)

In [20]:
ztest(count=np.array([1258103*0.1*(0.89-0.01),1258103*0.89]), nobs=np.array([1258103*0.1,1258103]))

(-10.76973702446419, 4.783655666578137e-27)

## Sequential Sampling

### Question: How many conversions are needed for a sequential A/B test?



序贯抽样方案是指在抽样时，不事先规定总的抽样个数（观测或实验次数），而是先抽少量样本，根据其结果，再决定停止抽样或继续抽样、抽多少，这样下去，直至决定停止抽样为止。


例如，一个产品抽样检验方案规定按批抽样品20件，若其中不合格品件数不超过3，则接收该批，否则拒收。在此，抽样个数20是预定的，是固定抽样。

若方案规定为：第一批抽出3个，若全为不合格品，拒收该批，若其中不合格品件数为 x1<3，则第二批再抽 3-x1 个，若全为不合格品，则拒收该批，若其中不合格品数为 x2<3-x1，则第三批再抽 3-x1-x2 个，这样下去，直到抽满20件或抽得3个不合格品为止。这是一个序贯抽样方案，其效果与前述固定抽样方案相同，但抽样个数平均讲要节省些。

## 2 Sample T-Test

### Question: Does the average value differ across two groups?

In [21]:
!pip install pingouin

In [22]:
# basic datascience/data manipulation libraries
import numpy as np
import pandas as pd
import numpy.random as npr
import scipy.stats as stats

# graphs
import seaborn as sns
import matplotlib.pyplot as plt

# formulat interface to statsmodels (standard linear models)
import statsmodels.formula.api as smf

# easy-to-use traditional psychological stats (t-test, anova)
import pingouin as pg

# hate these things
import warnings
warnings.filterwarnings("ignore")

In [23]:
# data
drug = np.array([101,100,102,104,102,97,105,105,98,101,100,123,105,103,100,95,102,106,
        109,102,82,102,100,102,102,101,102,102,103,103,97,97,103,101,97,104,
        96,103,124,101,101,100,101,101,104,100,101])
placebo = np.array([99,101,100,101,102,100,97,101,104,101,102,102,100,105,88,101,100,
           104,100,100,100,101,102,103,97,101,101,100,101,99,101,100,100,
           101,100,99,101,100,102,99,100,99])


# packing the data into a tidy dataframe can be nice
exp_df = pd.DataFrame(dict(group=[0]*len(drug)+[1]*len(placebo), score=np.r_[drug,placebo]))

exp_df.head()

,group,score
0,0,101
1,0,100
2,0,102
3,0,104
4,0,102


Is the mean of the drug group different than the mean of the placebo group?

In [24]:
print("Mean of drug group:", drug.mean())
print("Mean of placebo group:", placebo.mean())
print("The difference in means is: ", drug.mean()-placebo.mean())

Mean of drug group: 101.91489361702128
Mean of placebo group: 100.35714285714286
The difference in means is:  1.5577507598784166


The look close but not identical. However, "look" isn't enough.

Lets begin with a two-sample, independent samples t-test. We will assume that both groups have equal variance here.

In [25]:
pg.ttest(x=drug, y=placebo, paired=False, alternative='two-sided', correction=False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.558695,87,two-sided,0.122699,"[-0.43, 3.54]",0.330965,0.642,0.338035


We specified group `x` as `drug` and `y` as `placebo` (arbitrarily, you could flip that).

We used 'two-sided' which is the traditionally more conservative test which you use unless you have a strong a-priori belief one group is going to have a higher mean value.

We did not apply a correction known as the Welch-Satterthwaite correction for unequal variances.

We will try that later.

The results show that the t-value for the mean difference is 1.599.

The test has 87 degrees of freedom.

The p-value is 0.122699 which is greater than the traditional "alpha" cut off at p=0.05.

Therefore this test is not significant.

The 95% confidence interval for the differences between the means is -0.43 on the low end to 3.54 with (1.5577 the center).

The effect size (Cohen's d) is 0.331.

The Bayes Factor in favor of the alternative hypothesis (that the means are difference) is lower than one (0.642).

The post-hoc power of the test is 0.338.

All of this is consistent with there being basically no differences between these two groups.


## Survival Times

### Question: Does the hazard rate differ across two groups?

In [26]:
!pip install lifelines

In [27]:
from lifelines.datasets import load_waltons
from lifelines import KaplanMeierFitter
from lifelines.utils import median_survival_times
import matplotlib.pyplot as plt

# 数据载入
df = load_waltons()
print(df.head(),'\n')
print(df['T'].min(),df['T'].max(),'\n')
print(df['E'].value_counts(),'\n')
print(df['group'].value_counts(),'\n')

      T  E    group
0   6.0  1  miR-137
1  13.0  1  miR-137
2  13.0  1  miR-137
3  13.0  1  miR-137
4  19.0  1  miR-137 

6.0 75.0 

1    156
0      7
Name: E, dtype: int64 

control    129
miR-137     34
Name: group, dtype: int64 



可以看到数据有三列，其中T代表min(D,O)，其中D为死亡时间，O为观测截止时间。

E代表是否观察到“死亡”，1代表观测到了，0代表未观测到，即生存分析中的删失数据，共7个。

group代表是否存在病毒，miR-137代表存在病毒，control代表为不存在即对照组，根据统计，存在miR-137病毒人数34人，不存在129人。

需要注意，该格式并非严格的寿命表。

Logrank检验的零假设是指两组的生存时间分布完全一致，当我们通过计算拒绝零假设时，就可以认为两组的生存时间分布存在统计学差异。

In [28]:
from lifelines.statistics import logrank_test

T = df['T']
E = df['E']

dem = (df['group'] == 'miR-137')

results = logrank_test(T[dem], T[~dem], E[dem], E[~dem], alpha=.99)
results.print_summary()
print(results.p_value)
print(results.test_statistic)

<lifelines.StatisticalResult: logrank_test>
               t_0 = -1
 null_distribution = chi squared
degrees_of_freedom = 1
             alpha = 0.99
         test_name = logrank_test

---
 test_statistic      p  -log2(p)
         122.25 <0.005     91.99

2.0359832222854986e-28
122.2491255730062


P值小于0.05，拒绝原假设，存在差异。

## Count Data

### Question: Does the rate of arrival differ across two time periods?